This notebook is a slight modification of 
[LSTM by Keras with Unified Wi-Fi Feats](https://www.kaggle.com/kokitanisaka/lstm-by-keras-with-unified-wi-fi-feats/execution) by [@Kouki](https://www.kaggle.com/kokitanisaka). <br> 

Floor predictions are pretty accurate, so I wanted to test training the $x$ and $y$ coordinates with the floor data. The only thing I changed was training the $x$ and $y$ models with the floor data included. I add the floor predictions from [Simple 👌 99% Accurate Floor Model 💯](https://www.kaggle.com/nigelhenry/simple-99-accurate-floor-model) to the test data before making the $x$ and $y$ predictions.

## Overview

It demonstrates how to utilize [the unified Wi-Fi dataset](https://www.kaggle.com/kokitanisaka/indoorunifiedwifids).<br>
The Neural Net model is not optimized, there's much space to improve the score. 


In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from pathlib import Path
import glob
import pickle

import random
import os

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder

import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
import tensorflow.keras.backend as K
import tensorflow_addons as tfa
from tensorflow_addons.layers import WeightNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

### options
We can change the way it learns with these options. <br>
Especialy **NUM_FEATS** is one of the most important options. <br>
It determines how many features are used in the training. <br>
We have 100 Wi-Fi features in the dataset, but 100th Wi-Fi signal sounds not important, right? <br>
So we can use top Wi-Fi signals if we think we need to. 

In [2]:
# options

N_SPLITS = 10

SEED = 2021

NUM_FEATS = 20 # number of features that we use. there are 100 feats but we don't need to use all of them

base_path = '/kaggle'

In [3]:
def set_seed(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    session_conf = tf.compat.v1.ConfigProto(
        intra_op_parallelism_threads=1,
        inter_op_parallelism_threads=1
    )
    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    tf.compat.v1.keras.backend.set_session(sess)
    
def comp_metric(xhat, yhat, fhat, x, y, f):
    intermediate = np.sqrt(np.power(xhat-x, 2) + np.power(yhat-y, 2)) + 15 * np.abs(fhat-f)
    return intermediate.sum()/xhat.shape[0]

In [4]:
feature_dir = f"{base_path}/input/indoorunifiedwifids"
train_files = sorted(glob.glob(os.path.join(feature_dir, '*_train.csv')))
test_files = sorted(glob.glob(os.path.join(feature_dir, '*_test.csv')))
subm = pd.read_csv(f'{base_path}/input/indoor-location-navigation/sample_submission.csv', index_col=0)

In [5]:
with open(f'{feature_dir}/train_all.pkl', 'rb') as f:
  data = pickle.load( f)

with open(f'{feature_dir}/test_all.pkl', 'rb') as f:
  test_data = pickle.load(f)

In [6]:
# training target features

BSSID_FEATS = [f'bssid_{i}' for i in range(NUM_FEATS)]
RSSI_FEATS  = [f'rssi_{i}' for i in range(NUM_FEATS)]

In [7]:
# get numbers of bssids to embed them in a layer

wifi_bssids = []
for i in range(100):
    wifi_bssids.extend(data.iloc[:,i].values.tolist())
wifi_bssids = list(set(wifi_bssids))

wifi_bssids_size = len(wifi_bssids)
print(f'BSSID TYPES: {wifi_bssids_size}')

wifi_bssids_test = []
for i in range(100):
    wifi_bssids_test.extend(test_data.iloc[:,i].values.tolist())
wifi_bssids_test = list(set(wifi_bssids_test))

wifi_bssids_size = len(wifi_bssids_test)
print(f'BSSID TYPES: {wifi_bssids_size}')

wifi_bssids.extend(wifi_bssids_test)
wifi_bssids_size = len(wifi_bssids)

BSSID TYPES: 61206
BSSID TYPES: 33042


In [8]:
# preprocess

le = LabelEncoder()
le.fit(wifi_bssids)
le_site = LabelEncoder()
le_site.fit(data['site_id'])

ss = StandardScaler()
ss.fit(data.loc[:,RSSI_FEATS+['floor']])

StandardScaler()

In [9]:
data.loc[:,RSSI_FEATS+['floor']] = ss.transform(data.loc[:,RSSI_FEATS+['floor']])
for i in BSSID_FEATS:
    data.loc[:,i] = le.transform(data.loc[:,i])
    data.loc[:,i] = data.loc[:,i] + 1
    
data.loc[:, 'site_id'] = le_site.transform(data.loc[:, 'site_id'])

data.loc[:,RSSI_FEATS+['floor']] = ss.transform(data.loc[:,RSSI_FEATS+['floor']])

Add floor predictions.

In [10]:
simple_accurate_99 = pd.read_csv('../input/simple-99-accurate-floor-model/submission.csv') 

In [11]:
test_data['floor'] = simple_accurate_99['floor'].values

In [12]:
test_data.loc[:,RSSI_FEATS+['floor']] = ss.transform(test_data.loc[:,RSSI_FEATS+['floor']])
for i in BSSID_FEATS:
    test_data.loc[:,i] = le.transform(test_data.loc[:,i])
    test_data.loc[:,i] = test_data.loc[:,i] + 1
    
test_data.loc[:, 'site_id'] = le_site.transform(test_data.loc[:, 'site_id'])

test_data.loc[:,RSSI_FEATS+['floor']] = ss.transform(test_data.loc[:,RSSI_FEATS+['floor']])

In [13]:
site_count = len(data['site_id'].unique())
data.reset_index(drop=True, inplace=True)

In [14]:
set_seed(SEED)

## The model
The first Embedding layer is very important. <br>
Thanks to the layer, we can make sense of these BSSID features. <br>
<br>
We concatenate all the features and put them into LSTM. <br>
<br>
If something is theoritically wrong, please correct me. Thank you in advance. 

In [15]:
def create_model(input_data):

    # bssid feats
    input_dim = input_data[0].shape[1]

    input_embd_layer = L.Input(shape=(input_dim,))
    x1 = L.Embedding(wifi_bssids_size, 64)(input_embd_layer)
    x1 = L.Flatten()(x1)

    # rssi feats
    input_dim = input_data[1].shape[1]

    input_layer = L.Input(input_dim, )
    x2 = L.BatchNormalization()(input_layer)
    x2 = L.Dense(NUM_FEATS * 64, activation='relu')(x2)

    # site
    input_site_layer = L.Input(shape=(1,))
    x3 = L.Embedding(site_count, 1)(input_site_layer)
    x3 = L.Flatten()(x3)

    # main stream
    x = L.Concatenate(axis=1)([x1, x3, x2])

    x = L.BatchNormalization()(x)
    x = L.Dropout(0.3)(x)
    x = L.Dense(256, activation='relu')(x)

    x = L.Reshape((1, -1))(x)
    x = L.BatchNormalization()(x)
    #x = L.LSTM(128, dropout=0.3, recurrent_dropout=0.3, return_sequences=True, activation='relu')(x)
    x = L.LSTM(256, dropout=0.3, recurrent_dropout=0.3, return_sequences=True, activation='relu')(x)
    x = L.LSTM(16, dropout=0.1, return_sequences=False, activation='relu')(x)

    
    output_layer_1 = L.Dense(2, name='xy')(x)
    #output_layer_2 = L.Dense(1, activation='softmax', name='floor')(x)

    model = M.Model([input_embd_layer, input_layer, input_site_layer], 
                    [output_layer_1])

    model.compile(optimizer=tf.optimizers.Adam(lr=0.001),
                  loss='mse', metrics=['mse'])

    return model

In [16]:
score_df = pd.DataFrame()
predictions = list()

preds_x, preds_y = 0, 0
preds_f_arr = np.zeros((test_data.shape[0], N_SPLITS))

for fold, (trn_idx, val_idx) in enumerate(StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED).split(data.loc[:, 'path'], data.loc[:, 'path'])):
    X_train = data.loc[trn_idx, BSSID_FEATS + RSSI_FEATS + ['floor','site_id']]
    y_trainx = data.loc[trn_idx, 'x']
    y_trainy = data.loc[trn_idx, 'y']
    y_trainf = data.loc[trn_idx, 'floor']

    tmp = pd.concat([y_trainx, y_trainy], axis=1)
    #y_train = [tmp, y_trainf]
    y_train = tmp

    X_valid = data.loc[val_idx, BSSID_FEATS + RSSI_FEATS + ['floor','site_id']]
    y_validx = data.loc[val_idx, 'x']
    y_validy = data.loc[val_idx, 'y']
    y_validf = data.loc[val_idx, 'floor']

    tmp = pd.concat([y_validx, y_validy], axis=1)
    #y_valid = [tmp, y_validf]
    y_valid = tmp

    model = create_model([X_train.loc[:,BSSID_FEATS], X_train.loc[:,RSSI_FEATS+['floor']], X_train.loc[:,'site_id']])
    model.fit([X_train.loc[:,BSSID_FEATS], X_train.loc[:,RSSI_FEATS+['floor']], X_train.loc[:,'site_id']], y_train, 
                validation_data=([X_valid.loc[:,BSSID_FEATS], X_valid.loc[:,RSSI_FEATS+['floor']], X_valid.loc[:,'site_id']], y_valid), 
                batch_size=128, epochs=1000,
                callbacks=[
                ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, min_delta=1e-4, mode='min')
                , ModelCheckpoint(f'{base_path}/RNN_{SEED}_{fold}.hdf5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='min')
                , EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=5, mode='min', baseline=None, restore_best_weights=True)
            ])

    model.load_weights(f'{base_path}/RNN_{SEED}_{fold}.hdf5')
    #val_pred = model.predict([X_valid.loc[:,BSSID_FEATS], X_valid.loc[:,RSSI_FEATS], X_valid.loc[:,'site_id'], X_valid.loc[:,'floor']])

    pred = model.predict([test_data.loc[:,BSSID_FEATS], test_data.loc[:,RSSI_FEATS+['floor']], test_data.loc[:,'site_id']]) # test_data.iloc[:, :-1])
    preds_x += pred[:,0]
    preds_y += pred[:,1]
    #preds_f_arr[:, fold] = pred[1][:,0].astype(int)

    

    break # for demonstration, run just one fold as it takes much time.

preds_x /= (fold + 1)
preds_y /= (fold + 1)
    
print("*+"*40)
print("*+"*40)

#preds_f_mode = stats.mode(preds_f_arr, axis=1)
#preds_f = preds_f_mode[0].astype(int).reshape(-1)
preds_f = test_data['floor']
test_preds = pd.DataFrame(np.stack((preds_f, preds_x, preds_y))).T
test_preds.columns = subm.columns
test_preds.index = test_data["site_path_timestamp"]
test_preds["floor"] = test_preds["floor"].astype(int)
predictions.append(test_preds)

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


Epoch 1/1000
1815/1815 [==============================] - 169s 90ms/step - loss: 2420.4434 - mse: 2420.4434 - val_loss: 81.1210 - val_mse: 81.1210
Epoch 2/1000
1815/1815 [==============================] - 163s 90ms/step - loss: 184.6243 - mse: 184.6243 - val_loss: 65.2110 - val_mse: 65.2110
Epoch 3/1000
1815/1815 [==============================] - 164s 90ms/step - loss: 148.7120 - mse: 148.7120 - val_loss: 55.7797 - val_mse: 55.7797
Epoch 4/1000
1815/1815 [==============================] - 166s 91ms/step - loss: 125.7409 - mse: 125.7409 - val_loss: 49.8325 - val_mse: 49.8325
Epoch 5/1000
1815/1815 [==============================] - 167s 92ms/step - loss: 110.5230 - mse: 110.5230 - val_loss: 42.8975 - val_mse: 42.8975
Epoch 6/1000
1815/1815 [==============================] - 163s 90ms/step - loss: 96.3122 - mse: 96.3122 - val_loss: 42.0124 - val_mse: 42.0124
Epoch 7/1000
1815/1815 [==============================] - 162s 89ms/step - loss: 84.9532 - mse: 84.9532 - val_loss: 38.4090 - val_

In [17]:
all_preds = pd.concat(predictions)
all_preds = all_preds.reindex(subm.index)

## Fix the floor prediction
So far, it is not successfully make the "floor" prediction part with this dataset. <br>
To make it right, we can incorporate [@nigelhenry](https://www.kaggle.com/nigelhenry/)'s [excellent work](https://www.kaggle.com/nigelhenry/simple-99-accurate-floor-model). <br>

In [18]:
simple_accurate_99 = pd.read_csv('../input/simple-99-accurate-floor-model/submission.csv')

all_preds['floor'] = simple_accurate_99['floor'].values

In [19]:
all_preds.to_csv('submission.csv')